# 2024 Coding Challenge: Automated Object Detection and Counting

**Objective:** Develop a Python-based application that uses computer vision techniques to count and classify different objects from a set of images. The system should be able to differentiate at least three types of objects (e.g., cars, bicycles, pedestrians) in diverse lighting and background conditions. 

**Resources:** 
- https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#clearml-logging-and-automation-new
- https://github.com/ultralytics/yolov5/blob/master/tutorial.ipynb

## Data Preparation

1. Download Data:
- Run the download_data.py script to download the dataset:

### 1. Download Data:

In [1]:
# run 'download_data.py script' to download and get COCO dataset
# examine 5 images
%run ./../scripts/download_data.py

**Transfer to google drive**

In [ ]:
!cp -r data/raw/ "/Users/taishajoseph/Library/CloudStorage/GoogleDrive-taishavj28@gmail.com/My Drive/computer-vision-project/data/raw/"

In [ ]:
# Do not change this code
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!mkdir /content/data/
!mkdir /content/data/filtered/

In [ ]:
# Helper functions
def generate_direct_download_link(shared_link):
    """
    Generates a direct download link from a shared Google Drive link.

    Args:
        shared_link (str): The shared Google Drive link.

    Returns:
        str: The direct download link or an error message.

    Raises:
        ValueError: If the provided link is not a valid Google Drive shared link.
    """
    try:
        # Check if the shared link is a valid Google Drive link
        if "drive.google.com/file/d/" not in shared_link:
            raise ValueError("The provided link is not a valid Google Drive shared link.")
        
        # Extract the file ID from the shared link
        file_id = shared_link.split('/d/')[1].split('/')[0]
        
        # Create the direct download link
        direct_download_link = f"https://drive.google.com/uc?export=download&id={file_id}"
        
        return direct_download_link
    except Exception as e:
        return f"An error occurred: {e}"


In [ ]:
val2017 = "https://drive.google.com/file/d/18rji_6JgJYIBfv7EY2PQP3nQA689ZR2I/view?usp=sharing"
train2017 = "https://drive.google.com/file/d/1RgUrTkclcqtT8FVl912XCzX3jIahQwBU/view?usp=sharing"
annotations_trainval2017 = "https://drive.google.com/file/d/10oTb3FBJbWz7ZKjw2G-udamNMCBfYtuF/view?usp=sharing"

In [ ]:
# turn shared google drive shared link to direct download link
train2017 = generate_direct_download_link(train2017)
val2017 = generate_direct_download_link(val2017)
annotations_trainval2017 = generate_direct_download_link(annotations_trainval2017)

In [ ]:
#export zip data files from google drive
!gdown {train2017} -O /tmp/train2017.zip
!gdown {val2017} -O /tmp/val2017.zip
!gdown {annotations_trainval2017} -O /tmp/annotations_trainval2017.zip

In [ ]:
# Extract Data
import os
import zipfile

local_zip = '/tmp/train2017.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
local_zip = '/tmp/val2017.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
local_zip = '/tmp/test.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/annotations_trainval2017.zip')
zip_ref.close()

### 2. Prep Data:

1.1. Filter COCO Annotations
- We'll create a Python script to filter the annotations for the classes person, car, and bicycle. 
- In this section, we will update the annotations and copy the relevant images to a new directory.

- *Load the COCO annotations using COCO(input_annotation_file).*

    - Get category IDs for the classes of interest using getCatIds(catNms=categories_to_keep).
    - Get image IDs associated with the filtered categories using getImgIds(catIds=category_ids).

- *Filtering Process:*

    - Iterate over the filtered image IDs.
    - Load image information and corresponding annotations.
    - Append filtered images and annotations to the respective lists.
    - Copy relevant images to the output directory.

- *Saving Filtered Data:*

    - Save the filtered images, annotations, and categories to the output annotation file in COCO format.

In [5]:
import os
import shutil
import json
from pycocotools.coco import COCO

def ensure_dir_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)

def filter_coco_annotations(input_annotation_file, output_annotation_file, image_dir, output_image_dir, categories_to_keep):
    coco = COCO(input_annotation_file)
    category_ids = coco.getCatIds(catNms=categories_to_keep)
    image_ids = coco.getImgIds(catIds=category_ids)
    
    filtered_annotations = []
    filtered_images = []

    ensure_dir_exists(output_image_dir)
    ensure_dir_exists(os.path.dirname(output_annotation_file))

    for img_id in image_ids:
        img_info = coco.loadImgs(img_id)[0]
        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=category_ids)
        anns = coco.loadAnns(ann_ids)

        filtered_images.append(img_info)
        filtered_annotations.extend(anns)

        # Copy image to output directory
        src_img_path = os.path.join(image_dir, img_info['file_name'])
        dst_img_path = os.path.join(output_image_dir, img_info['file_name'])
        shutil.copy(src_img_path, dst_img_path)
    
    filtered_data = {
        'images': filtered_images,
        'annotations': filtered_annotations,
        'categories': [cat for cat in coco.loadCats(category_ids)]
    }

    with open(output_annotation_file, 'w') as f:
        json.dump(filtered_data, f)

In [6]:
categories_to_keep = ['person', 'car', 'bicycle']

filter_coco_annotations(
    './../data/raw/coco_annotations/annotations/instances_train2017.json',
    './../data/filtered/annotations/filtered_instances_train2017.json',
    './../data/raw/coco_train2017/train2017',
    './../data/filtered/images/train2017',
    categories_to_keep
)

loading annotations into memory...
Done (t=31.63s)
creating index...
index created!


In [8]:
filter_coco_annotations(
    './../data/raw/coco_annotations/annotations/instances_val2017.json',
    './../data/filtered/annotations/filtered_instances_val2017.json',
    './../data/raw/coco_val2017/val2017/val2017',
    './../data/filtered/images/val2017',
    categories_to_keep
)

loading annotations into memory...
Done (t=0.77s)
creating index...
index created!


2.1. Organize Dataset Directory
After filtering, we'll organize the dataset directory structure to be compatible with YOLOv5.
Right now, the dataset directory should look like this:

- data/
  - filtered/
    - images/
      - train2017/
      - val2017/
    - annotations/
      - filtered_instances_train2017.json
      - filtered_instances_val2017.json

2.2. Convert COCO Annotations to YOLO Format
We need to convert the filtered COCO annotations to YOLO format (i.e., .txt files for each image).

In [9]:
def convert_coco_to_yolo(annotations_file, labels_dir):
    # Load the COCO annotations file
    with open(annotations_file, 'r') as f:
        data = json.load(f)

    # Create a dictionary to map category IDs to category names
    categories = {cat['id']: cat['name'] for cat in data['categories']}
    
    # Create a dictionary to map category names to YOLO class indices
    category_to_index = {name: index for index, name in enumerate(categories.values())}

    # Ensure the labels directory exists
    os.makedirs(labels_dir, exist_ok=True)

    # Iterate over all annotations in the COCO dataset
    for ann in data['annotations']:
        image_id = ann['image_id']
        category_id = ann['category_id']
        bbox = ann['bbox']
        category_name = categories[category_id]

        # Skip categories that are not in the category_to_index dictionary
        if category_name not in category_to_index:
            continue

        # Get image information to calculate normalized bounding box coordinates
        image_info = next(img for img in data['images'] if img['id'] == image_id)
        image_width = image_info['width']
        image_height = image_info['height']

        # Calculate YOLO format coordinates (normalized)
        x_center = (bbox[0] + bbox[2] / 2) / image_width
        y_center = (bbox[1] + bbox[3] / 2) / image_height
        width = bbox[2] / image_width
        height = bbox[3] / image_height

        # Create the YOLO label string
        yolo_label = f"{category_to_index[category_name]} {x_center} {y_center} {width} {height}\n"

        # Determine the label file path based on the image file name (without extension)
        label_file_path = os.path.join(labels_dir, f"{image_info['file_name'].split('.')[0]}.txt")
        
        # Append the YOLO label to the label file
        with open(label_file_path, 'a') as label_file:
            label_file.write(yolo_label)

In [11]:
convert_coco_to_yolo(
    './../data/filtered/annotations/filtered_instances_train2017.json',
    # 'data/filtered/images/train2017',
    './../data/filtered/labels/train2017'
)

In [12]:
convert_coco_to_yolo(
    './../data/filtered/annotations/filtered_instances_val2017.json',
    # 'data/filtered/images/val2017',
    './../data/filtered/labels/val2017'
)

Right now, the dataset directory should look like this:

- data/
  - filtered/
    - images/
      - train2017/
      - val2017/
    - annotations/
      - filtered_instances_train2017.json
      - filtered_instances_val2017.json
    - labels/
      - train2017/
      - val2017/

**Before moving on to training a model:**

Step 1: Setting Up the Environment

- clone YOLOv5 repository
- install requirements

In [ ]:
# !git clone https://github.com/ultralytics/yolov5
# !cd yolov5
# %pip install -r requirements.txt

Step 3: YOLOv5 Data Configuration
- Create a YOLOv5 data configuration file titled: "coco_person_car_bicycle.yaml"
- cp configuration file to base directory of YOLOv5 repo where the train.py script is

In [ ]:
# !cp coco_person_car_bicycle.yaml ~/Documents/Projects/yolov5/coco_person_car_bicycle.yaml

## Checkpoint 1: Clear Memory

In [ ]:
# Clear variables
%reset -f

# Import necessary libraries
import gc

# Run garbage collector
gc.collect()

## Train
**2 Options for training a model with YOLOv5:**

- Fine-tune a pre-trained model with pre-exisiting weights
- Train a model from scratch

### Train Model (smallest one is yolov5n) from scratch
Should be in *~/Documents/Projects/yolov5/* directory

- Navigate to the models folder in the cloned yolov5 repo
- Modify the "yolov5n.yaml" file by replacing the number of classes from 80 to 3
- Adjust other parameters for each layer of the model as needed. (Let's start with the default parameters)

**Define parameters for training:**

- img 640: Use 640x640 images for training.
- batch 16: Use a batch size of 16.
- epochs 50: Train for 50 epochs.
- data coco_person_car_bicycle.yaml: Use custom data configuration file with fewer classes.
- weights '': Train from scratch by not using any pre-trained weights.
- cfg yolov5n.yaml: Use the configuration file for YOLOv5n (the smallest model). --> "models/yolov5n.yaml"
- device 0: Use the first GPU for training.
--project:Directory where results will be saved. ("/Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/")

**Recommended:** run in terminal

**If running in Jupyter notebook, be sure to clear the notebook memory first**

In [13]:
import os
#Define parameters for training:
data_config="coco_person_car_bicycle.yaml"
config="yolov5n.yaml"
out_dir = "/Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/train/"
os.makedirs(out_dir, exist_ok=True)

In [ ]:
# #example run in terminal (don't run):
# cd .. #navigate to computer vision project root directory

# mkdir yolov5n #create yolov5n directory

# mkdir /Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/train/ #create train subdirectory

# cd ~/Documents/Projects/yolov5 #navigate to yolov5 project directory

# python train.py --img 640 --batch 16 --epochs 50 --data coco_person_car_bicycle.yaml --weights '' --cfg yolov5n.yaml --device 0 --project /Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/train/

In [ ]:
#run as subprocess
!cd ~/Documents/Projects/yolov5
%run train.py --img 640 --batch 16 --epochs 50 --data coco_person_car_bicycle.yaml --weights '' --cfg {config} --device cpu --project {out_dir}

In [ ]:
#finetune pre-trained model
# python train.py --img 640 --batch 8 --epochs 10 --data coco_person_car_bicycle.yaml --weights yolov5s.pt --cfg yolov5n.yaml --device cpu --project /Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/train/

## Checkpoint 2: Clear Memory

In [ ]:
# Clear variables
%reset -f

# Import necessary libraries
import gc

# Run garbage collector
gc.collect()

## Evaluate
Should be in *~/Documents/Projects/yolov5/* directory

**Recommended:** run in terminal

**If running in Jupyter notebook, be sure to clear the notebook memory first**


**Evaluate the trained model on the validation dataset:**

In [ ]:
import os

out_dir = "/Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/train/"
os.makedirs(out_dir, exist_ok=True)
data_config="coco_person_car_bicycle.yaml"
wts = os.path.join(out_dir, "exp/weights/best.pt")
wts

In [ ]:
# !python val.py --data "coco_person_car_bicycle.yaml" --weights "runs/train/exp/weights/best.pt" --cache

In [ ]:
#run as subprocess
!cd ~/Documents/Projects/yolov5
%run val.py --data {data_config} --weights {wts}

## Checkpoint 3: Clear Memory

In [ ]:
# Clear variables
%reset -f

# Import necessary libraries
import gc

# Run garbage collector
gc.collect()

## Visualize Results

In [ ]:
out_dir = "/Users/taishajoseph/Documents/Projects/computer-vision-project/yolov5n/train/"

In [ ]:
import os
results = os.path.join(out_dir, "results.csv")
results

In [ ]:
!cd ~/Documents/Projects/yolov5
from utils.plots import plot_results

plot_results(results)  # plot 'results.csv' as 'results.png'